# Fraud Detection with XGBoost

In this example notebook, we will go step-by-step through the process of training and deploying an XGBoost fraud detection model using Triton's new FIL backend. Along the way, we'll show how to analyze the performance of a model deployed in Triton and optimize its performance based on specific SLA targets or other considerations. You can use Jupyter Lab running on the PyTorch container. If you haven't already installed the the PyTorch see [Set up PyTorch and Triton Containers]().

## Training Data
For this example, we will make use of data from the [IEEE-CIS Fraud Detection](https://www.kaggle.com/c/ieee-fraud-detection/overview) Kaggle competition.

In [22]:
#!tar xzvf data/train_transaction.tgz -C data/
train_csv = 'data/train_transaction.csv'

## Model Training
While the IEEE-CIS Kaggle competition focused on a more sophisticated problem involving analysis of both fraudulent transactions and the users linked to those transactions, we will use a simpler version of that problem (identifying fraudulent transactions only) to build our example model. In the following steps, we make use of cuML's preprocessing tools to clean the data and then train two example models using XGBoost.

In [23]:
import cudf
import cupy as cp
from cuml.preprocessing import SimpleImputer
from cuml.preprocessing import LabelEncoder
# Due to an upstream bug, cuML's train_test_split function is
# currently non-deterministic. We will therefore use sklearn's
# train_test_split in this example to obtain more consistent
# results.
from sklearn.model_selection import train_test_split

SEED=0

In [24]:
# Load data from CSV files into cuDF DataFrames
data = cudf.read_csv(train_csv)

In [25]:
# Replace NaNs in data
nan_columns = data.columns[data.isna().any().to_pandas()]
float_nan_subset = data[nan_columns].select_dtypes(include='float64')

imputer = SimpleImputer(missing_values=cp.nan, strategy='median')
data[float_nan_subset.columns] = imputer.fit_transform(float_nan_subset)

obj_nan_subset = data[nan_columns].select_dtypes(include='object')
data[obj_nan_subset.columns] = obj_nan_subset.fillna('UNKNOWN')

In [26]:
# Convert string columns to categorical or perform label encoding
cat_columns = data.select_dtypes(include='object')
for col in cat_columns.columns:
    data[col] = LabelEncoder().fit_transform(data[col])

In [27]:
# Split data into training and testing sets
X = data.drop('isFraud', axis=1)
y = data.isFraud.astype(int)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y.to_numpy(), random_state=SEED)

#X_train, X_test, y_train, y_test = train_test_split(
#    X.to_pandas(), y.to_pandas(), test_size=0.3, stratify=y.to_pandas(), random_state=SEED
#)
# Copy data to avoid slowdowns due to fragmentation
X_train = X_train.copy()
X_test = X_test.copy()

In [28]:
import xgboost as xgb

In [30]:
# Define model training function
def train_model(num_trees, max_depth):
    model = xgb.XGBClassifier(
        tree_method='gpu_hist',
        predictor='gpu_predictor',
        #tree_method='hist',
        #predictor='cpu_predictor',
        use_label_encoder=False,
        eval_metric='aucpr',
        objective='binary:logistic',
        max_depth=max_depth,
        n_estimators=num_trees
    )
    model.fit(
        X_train,
        y_train,
        eval_set=[(X_test, y_test)]
    )
    return model
# Train a small model with just 500 trees and a maximum depth of 3
model = train_model(300, 3)

[0]	validation_0-aucpr:0.27398
[1]	validation_0-aucpr:0.31137
[2]	validation_0-aucpr:0.36501
[3]	validation_0-aucpr:0.37410
[4]	validation_0-aucpr:0.38408
[5]	validation_0-aucpr:0.41039
[6]	validation_0-aucpr:0.41761
[7]	validation_0-aucpr:0.42502
[8]	validation_0-aucpr:0.43361
[9]	validation_0-aucpr:0.44091
[10]	validation_0-aucpr:0.44511
[11]	validation_0-aucpr:0.44798
[12]	validation_0-aucpr:0.45469
[13]	validation_0-aucpr:0.45892
[14]	validation_0-aucpr:0.46160
[15]	validation_0-aucpr:0.46531
[16]	validation_0-aucpr:0.46899
[17]	validation_0-aucpr:0.47269
[18]	validation_0-aucpr:0.47758
[19]	validation_0-aucpr:0.48164
[20]	validation_0-aucpr:0.48452
[21]	validation_0-aucpr:0.48778
[22]	validation_0-aucpr:0.49041
[23]	validation_0-aucpr:0.49339
[24]	validation_0-aucpr:0.49637
[25]	validation_0-aucpr:0.49816
[26]	validation_0-aucpr:0.50005
[27]	validation_0-aucpr:0.50383
[28]	validation_0-aucpr:0.50801
[29]	validation_0-aucpr:0.51021
[30]	validation_0-aucpr:0.51175
[31]	validation_0-